In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

from cardinality_estimation.featurizer import Featurizer
from query_representation.query import load_qrep

import glob
import random
import os
import json
import time

# Setup file paths / Download query data

In [ ]:
import errno
def make_dir(directory):
    try:
        os.makedirs(directory)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

In [ ]:
# TODO
TRAINDIR = os.path.join(os.path.join("", "queries"), "mlsys1-train")
VALDIR = os.path.join(os.path.join("", "queries"), "mlsys1-val")
TESTDIR = os.path.join(os.path.join("", "queries"), "mlsys1-test")

RESULTDIR = os.path.join("", "results")
make_dir(RESULTDIR)

# Query loading helper functions

In [ ]:

def load_qdata(fns):
    qreps = []
    for qfn in fns:
        qrep = load_qrep(qfn)
        # TODO: can do checks like no queries with zero cardinalities etc.
        qreps.append(qrep)
        template_name = os.path.basename(os.path.dirname(qfn))
        qrep["name"] = os.path.basename(qfn)
        qrep["template_name"] = template_name
    return qreps

def get_query_fns(basedir, template_fraction=1.0):
    fns = []
    tmpnames = list(glob.glob(os.path.join(basedir, "*")))
    assert template_fraction <= 1.0
    
    for qi,qdir in enumerate(tmpnames):
        if os.path.isfile(qdir):
            continue
        template_name = os.path.basename(qdir)
        # let's first select all the qfns we are going to load
        qfns = list(glob.glob(os.path.join(qdir, "*.pkl")))
        qfns.sort()
        num_samples = max(int(len(qfns)*template_fraction), 1)
        random.seed(1234)
        qfns = random.sample(qfns, num_samples)
        fns += qfns
    return fns

# Evaluation helper functions

In [ ]:
def get_preds(alg, qreps):
    if isinstance(qreps[0], str):
        # only file paths sent
        qreps = load_qdata(qreps)
    
    ests = alg.test(qreps)
    return ests

def eval_alg(alg, eval_funcs, qreps, samples_type, result_dir="./results/"):
    '''
    '''
    np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

    alg_name = alg.__str__()
    exp_name = alg.get_exp_name()
    
    if isinstance(qreps[0], str):
        # only file paths sent
        qreps = load_qdata(qreps)
    
    ests = alg.test(qreps)

    for efunc in eval_funcs:
        rdir = None
        if result_dir is not None:
            rdir = os.path.join(result_dir, exp_name)
            make_dir(rdir)

        errors = efunc.eval(qreps, ests, samples_type=samples_type,
                result_dir=rdir,
                num_processes = -1,
                alg_name = alg_name)

        print("{}, {}, #samples: {}, {}: mean: {}, median: {}, 99p: {}"\
                .format(samples_type, alg, len(errors),
                    efunc.__str__(),
                    np.round(np.mean(errors),3),
                    np.round(np.median(errors),3),
                    np.round(np.percentile(errors,99),3)))

# Load queries

In [ ]:
# set template_fraction <= 1.0 to test quickly w/ smaller datasets
train_qfns = get_query_fns(TRAINDIR, template_fraction = 0.1)
val_qfns = get_query_fns(VALDIR, template_fraction = 1.0)
test_qfns = get_query_fns(TESTDIR, template_fraction = 1.0)
#trainqs = load_qdata(train_qfns)
## can take up a lot of memory, so we avoid loading query data if not needed.
#valqs = load_qdata(val_qfns)
#testqs = load_qdata(test_qfns)

print("Selected {} training queries, {} validation queries, {} test queries".\
      format(len(train_qfns), len(val_qfns), len(test_qfns)))

# What is cardinality estimation and query optimization?

We are going to use a simple / small query for visualizing / illustrating the key properties of our tasks. These visualization functions work for larger queries, but we'll need to play around with matplotlib to make them look fine; it gets too large to visualize properly for most CEB queries.

### Simple example query

* Queries in CEB are stored in *.pkl file because it contains more than just the SQL; Among other things, each query contains cardinality (size) estimates for all <it> subplans </it> of the query. These subplans are the intermediate results encountered by an optimizer when joining and optimizing the query (shown later).
* The loaded file is a dictionary; Next we access the SQL:

In [ ]:
qfn = os.path.join(os.path.join("queries", "example_queries"), "2.pkl")
qrep = load_qdata([qfn])[0]
print(qrep["name"])
print(qrep["sql"])

### Join Graph, Cardinality Estimation and Query Optimization

In [ ]:
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
from query_representation.viz import draw_plan_graph
#matplotlib.use('Agg') # no UI backendz

join_graph = qrep["join_graph"]
nx.draw(join_graph, pos=nx.spring_layout(join_graph), 
        with_labels=True, node_size=2000, font_size=24)
plt.show()

* The two key properties of a query we care about are the join graph, and each of its subplans --- which is any connected subgraph of the join graph.

* Note: each node in the join graph represents a table used in the SQL query above (the node label is the ALIAS used in the SQL, such as title as t); An edge in the join graph implies there is a condition such as t.id = ci.movie_id in the query above.

* The process of optimizing a query is essentially like contracting the join graph to a point of single node; at each step, you are allowed to combine (``join``) any two connected nodes. Any order of applying the joins is semantically an equivalent way to execute a SQL query. The job of an optimizer is to choose a fast way to execute the queries. More [details](https://bertwagner.com/posts/does-the-join-order-of-my-tables-matter/)

* The join graph does not show the query' filters / predicates. For instance, `rt.role IN ('actor', 'director')` in the SQL query above. IN CEB, we will have a few different join graphs, and then for each join graph, A LOT of queries in which we vary these filters --- which changes the sizes of each node in the join graph; These filters can also interact with each other after joins --- and how these sizes change is crucial for choosing between different orders of executing the joins.

* Cardinality estimation is about estimating sizes for all the subplans (i.e., connected subgraphs of the join graph). These subgraphs would be encountered by an optimizer when it considers a particular sequence of joins. Processing a large sized join can be expensive --- because that will involve more CPU, more memory usage, and so on; 

* Some join orders can be particularly bad. For instance, consider joining the tables `cast_info (ci)` and `title (t)` first in the query above. These are both very large tables; Joining them, creates an even larger intermediate results. Future join operations will have to process these intermediate rows. Instead, if you join `cast_info` with `role_type`, then as the filter predicate: `rt.role IN ('actor', 'director')` reduces the size of the `cast_info` table in a non-trivial way. And so on. 

* Note: If the filter on title (t.production_year <= 2015) was different, such as `t.production_year <= 1890`, which will mean there would be very few qualifying rows in the `title` table, then joining the `cast_info` and `title` tables in the first step may have been a good move!

### Subset Graph / Plan Graph

In [ ]:
# copying, because we will do changes to it for visualizing, such as adding a source node.
from cardinality_estimation.algs import TrueCardinalities
import copy

subsetg = copy.deepcopy(qrep["subset_graph"])
subsetg = subsetg.reverse()

nodes = list(subsetg.nodes())
nodes.sort(key=lambda x: len(x))
final_node = nodes[-1]

true_alg = TrueCardinalities()
y = true_alg.test([qrep])[0]

fig, ax = plt.subplots(1,1,figsize=(15,10))
SOURCE_NODE = tuple(["s",])

draw_plan_graph(subsetg, y, "C", ax=ax, source_node=SOURCE_NODE,
               final_node=final_node, font_size=20, width=.0, edge_color=None, 
               bold_opt_path=True)

* It is useful to represent all these subplans in a graph structure of its own. This lets us reason about query optimization in terms of standard graph theoretic constructs. In code, we refer to this as the subset_graph; Intuitively, it should be clear what each node and edges of this graph represent.

* Please see Section 4 of this [paper](https://vldb.org/pvldb/vol14/p2019-negi.pdf) for precise mathematical definitions of what we show here --- including how edges are defined etc. , and note that we referred to it as the plan graph in the paper. Another recent [interesting paper](https://arxiv.org/abs/2105.08878) referred to the same structure as cardinality estimation graphs.

* Each edge in this subset graph is a join in the query. An optimizer typically costs each join using various heuristics. We use a simple, but practically reasonable, cost model (referred to as `C` in the [paper](https://vldb.org/pvldb/vol14/p2019-negi.pdf)) --- these are shown in green-yellow-red colors going from cheaper to expensive operations (note: the costs are a function of cardinalities; we use true cardinalities to cost these edges). Each path from the top `S` node, to the bottom-most node represents a [left deep plan]
(https://www.mathcs.emory.edu/~cheung/Courses/554/Syllabus/5-query-opt/left-deep-trees.html) in the context of query optimization. Thus, the shortest path from the top node (S) to the bottom node (D) is equivalent to an optimizer finding the cheapest (left deep) plan (typically, using dynamic programming algorithms). The shortest path given the true cardinalities is highlighted.

### The task

* The task is to give a size estimate to each node in the subset graph shown above. Each node corresponds to a particular subgraph of the join graph, i.e., a particular sequence of joins. Thus, there is a corresponding SQL query, and a true label for this query (which we have generated by executing these queries). 

* The learning task is to take a representation of the SQL query associated with each node (featurizes the tables, joins, and filters in the query), and output a size estimate for it.

* For instance, we will show how we evaluate two baselines below --- the actual true cardinalities, and the estimates from PostgreSQL.

In [ ]:
from cardinality_estimation.algs import TrueCardinalities, Postgres
true_alg = TrueCardinalities()
pg_alg = Postgres()
y = true_alg.test([qrep])[0]
ypg = pg_alg.test([qrep])[0]

### Evaluation 1: Q-Error

* Q-Error = `max((y/yhat), (yhat/y))`
* This was defined by Moerkotte et al. in the [paper](http://www.vldb.org/pvldb/vol2/vldb09-657.pdf). It provides strong arguments for why this is the right way to evaluate cardinality estimates if we care about query plans (as opposed to error metrics like mean squared error). 

### Evaluation 2: Plan-Cost

* Here, we will take the estimates provided, cost each edge in the subset graph based on those estimates, and find the best plan based on these estimated costs. 

* As we see below, the plan chosen using PostgreSQL estimates was different from the plan chosen using true estimates. 

* The `true` cost of this plan (the cost of the edges using the true cardinalities) would be the plan-cost. Intuitively, this cost is a good proxy for how good / bad the plan is, or how long it will take to execute them (as long as the cost model we use reasonably represents the execution scenario).

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10))
#from query_representation.utils import SOURCE_NODE
SOURCE_NODE = tuple(["s",])
draw_plan_graph(subsetg, ypg, "C", ax=ax, source_node=SOURCE_NODE,
               final_node=final_node, font_size=20, width=.0, edge_color=None, 
               bold_opt_path=True)

plt.show()

# Evaluation Functions

* Now, that we have explained what these evaluation functions are, here we just import their implementations to be used for evaluating the learned models.
* SimplePlanCost is an approximation to a DBMS based plan cost (for instance, PostgresPlanCost --- implemented in evaluation.eval_fns as well). There are two simplifying assumptions in SimplePlanCost:
    * Only considers left deep plans
    * Cost model is quite simple simple; it only really makes sense in case of index nested loop joins. In reality, indices may not exist, other join kinds may be better etc. Even so, it is fairly correlated with Postgres Plan Cost in our experiments.

In [ ]:
from evaluation.eval_fns import QError, SimplePlanCost
EVAL_FNS = []
EVAL_FNS.append(QError())
EVAL_FNS.append(SimplePlanCost())

# Helper function for initializing featurizer

* Featurizer object contains information about the db, e.g., tables, joins, columns, how to featurize predicate filters etc.

In [ ]:
def init_featurizer(featurization_type):
    # Load database specific data, e.g., information about columns, tables etc.
    dbdata_fn = os.path.join(TRAINDIR, "dbdata.json")
    featurizer = Featurizer(None, None, None, None, None)
    with open(dbdata_fn, "r") as f:
        dbdata = json.load(f)
    featurizer.update_using_saved_stats(dbdata)

    # ynormalization: takes log(y) for all target values, y.
    featurizer.setup(ynormalization="log",
            featurization_type=featurization_type)
    featurizer.update_ystats(trainqs)
    return featurizer

In [ ]:
# going to start training the models
trainqs = load_qdata(train_qfns)

# Submission code

In [ ]:
def save_submission(predictions, filename):
    "Take model output & save for cardinality estimation benchmark upload."""
    np.save(filename, np.array([value for item in predictions for key, value in sorted(item.items())]))

# RandomForest model

In [ ]:
from cardinality_estimation.algs import RandomForest
featurizer = init_featurizer("combined")
rf = RandomForest(grid_search = False,
                n_estimators = 10,
                max_depth = 6)
rf.train(trainqs, valqs=None, testqs=None,
    featurizer=featurizer, result_dir=RESULTDIR)

In [ ]:
# evaluate model
eval_alg(rf, EVAL_FNS, trainqs, "train")
eval_alg(rf, EVAL_FNS, val_qfns, "val")

preds = get_preds(rf, test_qfns)

# Save submission file

In [ ]:
save_submission(preds, "mysubmisson.npy")

# XGBoost

In [ ]:
from cardinality_estimation.algs import XGBoost
featurizer = init_featurizer("combined")
xgb = XGBoost(grid_search=False, tree_method="hist",
                       subsample=1.0, n_estimators = 100,
                       max_depth=10, lr = 0.01)
xgb.train(trainqs, valqs=None, testqs=None,
    featurizer=featurizer, result_dir=RESULTDIR)

In [ ]:
# evaluate model
eval_alg(xgb, EVAL_FNS, trainqs, "train")
eval_alg(xgb, EVAL_FNS, valqs, "val")


# TODO: test set prdictions; should submit these for the leaderboard?
#preds = xgb.test(testqs)

# Fully Connected Neural Network / Multilayer Perceptron

In [ ]:
from cardinality_estimation.fcnn import FCNN
featurizer = init_featurizer("combined")
fcnn = FCNN(max_epochs = 10,
     lr=0.0001,
     mb_size = 512,
     weight_decay = 0.0,
     result_dir = "./results",
     num_hidden_layers=4,
     optimizer_name="adamw",
     clip_gradient=20.0,
     loss_func_name = "mse",
     hidden_layer_size = 256)

fcnn.train(trainqs, valqs=None, testqs=None,
    featurizer=featurizer, result_dir=RESULTDIR)

In [ ]:
# evaluate model
eval_alg(fcnn, EVAL_FNS, trainqs, "train")
eval_alg(fcnn, EVAL_FNS, valqs, "val")

# TODO: test set prdictions; should submit these for the leaderboard?
#preds = fcnn.test(testqs)

# Multi Set Convolutional Network

## Notes

* Introduced by Kipf et al. in this [paper](https://arxiv.org/abs/1809.00677). Architecture based on [Deep Sets](https://arxiv.org/abs/1703.06114).
* Does not reserve an exact mapping for features on a particular table / column. Treats table features, join features, and predicate features as set of vectors. Has practical benefits over the flat 1d featurization (see discussion in README). But requires each batch to have same shape; thus a lot of the smaller query features need to be padded with zeros, which makes the memory consumption become much larger (can probably improve this somehow).
* load_padded_mscn_feats = True (see MSCN initialization below), loads these padded sets in memory; takes more RAM, but is faster; load_padded_mscn_feats = False, pads the vectors as needed --- takes longer to train (TODO: current python implementation can be improved).

In [ ]:
from cardinality_estimation.mscn import MSCN

featurizer = init_featurizer("set")

# load_padded_mscn_feats = True means all the fea
mscn = MSCN(max_epochs = 10,
     load_padded_mscn_feats = False,
     lr=0.0001,
     mb_size = 512,
     weight_decay = 0.0,
     result_dir = "./results",
     optimizer_name="adamw",
     clip_gradient=20.0,
     loss_func_name = "mse",
     hidden_layer_size = 256)

mscn.train(trainqs, valqs=None, testqs=None,
    featurizer=featurizer, result_dir=RESULTDIR)

In [ ]:
# evaluate model
eval_alg(mscn, EVAL_FNS, trainqs, "train")
eval_alg(mscn, EVAL_FNS, valqs, "val")

# TODO: test set prdictions; should submit these for the leaderboard?
#preds = mscn.test(testqs)